In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn import linear_model
from sklearn import neighbors
from sklearn.neighbors import KNeighborsRegressor
from collections import Counter

# Data

Here we import that data from the CSV files and create a subset sample to do rapid training and validation cycles on.

In [2]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
# Store gap values
Y_train = df_train.gap.values

# Take a sample with training and validation sets
df_sample = df_train.sample(100000)
msk = np.random.rand(len(df_sample)) < 0.8

# Features

Here we extract features from the SMILES string and winnow down the existing features to create a more robust feature set.

In [4]:
def makeFeatures(df):
    
    ##########################
    ### DROP EMPTY COLUMNS ###
    ##########################
    # Remove 0 columns (columns with no data)
    zero_cols = []
    for i in range(1,257):
        if df['feat_%03d' % i].sum() == 0:
            zero_cols.append('feat_%03d' % i)
    df = df.drop(zero_cols, axis=1)
    
    
    ##############################
    ### SMILE CHARACTER COUNTS ###
    ##############################
    smiles = df.smiles
    smileydict = smiles.map(lambda x: dict(Counter(x)))
    smile_alphabet=list(set(''.join(smiles.iloc[0:50])))
    for smile in smile_alphabet:
        smilechar = smile
        if smile == '=':
            smilechar = 'equal'
        df['smile_'+smilechar] = smileydict.map(lambda x: x[smile] if smile in x.keys() else 0)
        
    
    ###########################
    ### FEATURE ENGINEERING ###
    ###########################
    #smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
    #df_all['smiles_len'] = pd.DataFrame(smiles_len)

    # Add length of smile
    df['smile_length'] = df.smiles.map(lambda x: len(x))

    # Add number of C's divided by length
    df['smile_percentc'] = (df.smile_c / df.smile_length)
    df['smile_percentC'] = (df.smile_C / df.smile_length)

    # Count specific molecules
    # [nH]
    df['smile_nh'] = df.smiles.map(lambda x: x.count('[nH]'))
    df['smile_si'] = df.smiles.map(lambda x: x.count('Si'))
    df['smile_sih2'] = df.smiles.map(lambda x: x.count('[SiH2]'))
    df['smile_se'] = df.smiles.map(lambda x: x.count('[se]'))
    df['smile_CdoubleC'] = df.smiles.map(lambda x: x.count('C=C'))
    df['smile_doubleC'] = df.smiles.map(lambda x: x.count('CC'))
    df['smile_doublec'] = df.smiles.map(lambda x: x.count('cc'))
    df['smile_triplec'] = df.smiles.map(lambda x: x.count('ccc'))
    df['smile_quadc'] = df.smiles.map(lambda x: x.count('cccc'))
    df['smile_quintc'] = df.smiles.map(lambda x: x.count('ccccc'))
    #df['smile_c2'] = df.smiles.map(lambda x: x.count('c2'))
    #df['smile_c3'] = df.smiles.map(lambda x: x.count('c3'))
    #df['smile_c4'] = df.smiles.map(lambda x: x.count('c4'))

    df['smile_C1equalCc2'] = df.smiles.map(lambda x: x.count('C1=Cc2'))
    df['smile_C1'] = df.smiles.map(lambda x: x.count('C1'))
    df['smile_c1'] = df.smiles.map(lambda x: x.count('c1'))
    df['smile_equalCCCequal'] = df.smiles.map(lambda x: x.count('=CCC='))
    df['smile_equalCCequal'] = df.smiles.map(lambda x: x.count('=CC='))
    df['smile_equalCequal'] = df.smiles.map(lambda x: x.count('=C='))
    df['smile_C1equalCCequalC'] = df.smiles.map(lambda x: x.count('C1=CC=C'))

    # Parentheses molecules
    df['smile_parenC1'] = df.smiles.map(lambda x: x.count('(C1)'))
    df['smile_parenc1'] = df.smiles.map(lambda x: x.count('(c1)'))
    df['smile_parencc1'] = df.smiles.map(lambda x: x.count('(cc1)'))
    df['smile_pareno1'] = df.smiles.map(lambda x: x.count('(o1)'))
    df['smile_parens1'] = df.smiles.map(lambda x: x.count('(s1)'))
    df['smile_parenccc4mol'] = df.smiles.map(lambda x: x.count('(ccc4=C[SiH2]C=c34)'))
    df['smile_parenccinnermol'] = df.smiles.map(lambda x: x.count('(cc(-c3ccco3)c3=CCC=c13)'))
    df['smile_parennegc3cco3'] = df.smiles.map(lambda x: x.count('(-c3ccco3)'))
    df['smile_parenncc3c12'] = df.smiles.map(lambda x: x.count('(ncc3c12)'))
    df['smile_parenccc34'] = df.smiles.map(lambda x: x.count('(ccc34)'))
    df['smile_parencc4ccc3c2cn1'] = df.smiles.map(lambda x: x.count('(cc4ccc3c2cn1)'))

    # Special
    df['smile_percent_aromatic'] = (df.smile_c + df.smile_o + df.smile_n + df.smile_s / df.smile_length)

    # Start
    df['smile_start_C1'] = df.smiles.map(lambda x: x.startswith('C1'))
    df['smile_start_C1equal'] = df.smiles.map(lambda x: x.startswith('C1='))
    df['smile_start_c1'] = df.smiles.map(lambda x: x.startswith('c1'))
    df['smile_start_cc1'] = df.smiles.map(lambda x: x.startswith('cc1'))
    df['smile_start_c1sc'] = df.smiles.map(lambda x: x.startswith('c1sc'))
    df['smile_start_c1ccc'] = df.smiles.map(lambda x: x.startswith('c1ccc'))
    df['smile_start_nH'] = df.smiles.map(lambda x: x.startswith('[nH]'))
    df['smile_start_C1equalCCequalC'] = df.smiles.map(lambda x: x.startswith('C1=CC=C'))

    # End
    df['smile_end_c1ccc'] = df.smiles.map(lambda x: x.endswith('c1ccc'))
    df['smile_end_o1'] = df.smiles.map(lambda x: x.endswith('o1'))
    df['smile_end_ccsc12'] = df.smiles.map(lambda x: x.endswith('ccsc12'))

    #df['smile_percent_bond'] = df.smile_equal / df.smile_length
    
    
    ################################
    ### DROP UNNECESSARY COLUMNS ###
    ################################    
    df = df.drop('smile_length', axis=1)
    df = df.drop(['smiles'], axis=1)
    
    
    # Return the data frame with all of the new features added in
    return df

In [7]:
df_sample = makeFeatures(df_sample)
df_sample.head(5)

KeyError: 'feat_002'

In [79]:
# Create the training sample
df_sample_train = df_sample[msk]
Y_sample_train = df_sample_train.gap.values
df_sample_train = df_sample_train.drop(['gap'], axis=1)

# Create the testing sample
df_sample_test = df_sample[~msk]
Y_sample_test = df_sample_test.gap.values
df_sample_test = df_sample_test.drop(['gap'], axis=1)

print "Training Sample Shape", df_sample_train.shape
print "Y Training Sample Shape", Y_sample_train.shape
print "Test Sample Shape", df_sample_test.shape
print "Y Test Sample Shape", Y_sample_test.shape

Training Sample Shape (80033, 94)
Y Training Sample Shape (80033,)
Test Sample Shape (19967, 94)
Y Test Sample Shape (19967,)


# Models

Here we test out a series of models on the training and valida

In [80]:
# Sample Linear Regression Testing
LR = LinearRegression()

# Train the model using the training sets
LR.fit(df_sample_train, Y_sample_train)

# The coefficients
#print('Coefficients: \n', LR.coef_)
# RMSE
print("RMSE: %.3f" % np.sqrt(np.mean((LR.predict(df_sample_test) - Y_sample_test) ** 2)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.3f' % LR.score(df_sample_test, Y_sample_test))

RMSE: 0.207
Variance score: 0.740


In [21]:
# TOO SLOW!
# Sample Polynomial Interpolation Testing
#Poly = make_pipeline(PolynomialFeatures(degree), Ridge())
#Poly.fit(df_sample_train, Y_sample_train)

#print "RMSE: %.3f" % np.sqrt(np.mean((Poly.predict(df_sample_test) - Y_sample_test) ** 2))

In [12]:
# Sample Ridge Testing
Ridge = linear_model.Ridge(alpha = .001)

Ridge.fit(df_sample_train, Y_sample_train)

print("RMSE: %.3f" % np.sqrt(np.mean((Ridge.predict(df_sample_test) - Y_sample_test) ** 2)))
print('Variance score: %.3f' % Ridge.score(df_sample_test, Y_sample_test))

RMSE: 0.209
Variance score: 0.741


In [13]:
# Sample Lasso Testing
Lasso = linear_model.Lasso(alpha = .001)

Lasso.fit(df_sample_train, Y_sample_train)

print("RMSE: %.3f" % np.sqrt(np.mean((Lasso.predict(df_sample_test) - Y_sample_test) ** 2)))
print('Variance score: %.3f' % Lasso.score(df_sample_test, Y_sample_test))

RMSE: 0.213
Variance score: 0.729


/Users/Avery/anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


In [14]:
# KNN
KNN = neighbors.KNeighborsRegressor(n_neighbors=12)

KNN.fit(df_sample_train, Y_sample_train)

#print "N_neighbors: %d" % 12
print("RMSE: %.3f" % np.sqrt(np.mean((KNN.predict(df_sample_test) - Y_sample_test) ** 2)))
#print('Variance score: %.3f' % KNN.score(df_sample_test, Y_sample_test))

RMSE: 0.186


In [81]:
# Random Forest Testing
RF = RandomForestRegressor(n_estimators=40, min_samples_split=8)
RF.fit(df_sample_train, Y_sample_train)
print "Finished fitting"

print("RMSE: %.3f" % np.sqrt(np.mean((RF.predict(df_sample_test) - Y_sample_test) ** 2)))

Finished fitting
RMSE: 0.150


In [18]:
# Building an Ensemble
lr_predictions = LR.predict(df_sample_train)
ridge_predictions = Ridge.predict(df_sample_train)
lasso_predictions = Lasso.predict(df_sample_train)
knn_predictions = KNN.predict(df_sample_train)
rf_predictions = RF.predict(df_sample_train)


dfensemble=pd.DataFrame.from_dict({'lr':lr_predictions,
                                   'ridge':ridge_predictions,
                                   'lasso':lasso_predictions, 
                                   'knn':knn_predictions,
                                   'rf':rf_predictions,
                                   'y':Y_sample_train})

est = LinearRegression()
est.fit(dfensemble[['lr', 'ridge', 'lasso', 'knn', 'rf']].values, dfensemble['y'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
# Testing the Ensemble
lr_predictions_test = LR.predict(df_sample_test)
ridge_predictions_test = Ridge.predict(df_sample_test)
lasso_predictions_test = Lasso.predict(df_sample_test)
knn_predictions_test = KNN.predict(df_sample_test)
rf_predictions_test = RF.predict(df_sample_test)

dfensembletest=pd.DataFrame.from_dict({'lr':lr_predictions_test,
                                   'ridge':ridge_predictions_test,
                                   'lasso':lasso_predictions_test,
                                   'knn':knn_predictions_test,
                                   'rf':rf_predictions_test,
                                   'y':Y_sample_test})

epreds = est.predict(dfensembletest[['lr', 'ridge', 'lasso', 'knn', 'rf']].values)

print("RMSE: %.3f" % np.sqrt(np.mean((epreds - Y_sample_test) ** 2)))
#print('Variance score: %.3f' % est.score(df_sample_test, Y_sample_test))

RMSE: 0.156


# Prediction

In [8]:
# Clean up Actual Training and Testing Data
X_train = makeFeatures(df_train)
X_test = makeFeatures(df_test)

# Delete 'gap' column in training
X_train = X_train.drop(['gap'], axis=1)

# Delete 'Id' column in testing
X_test = X_test.drop(['Id'], axis=1)

print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Index([u'smiles', u'feat_001', u'feat_002', u'feat_003', u'feat_004',
       u'feat_005', u'feat_006', u'feat_007', u'feat_008', u'feat_009',
       ...
       u'feat_248', u'feat_249', u'feat_250', u'feat_251', u'feat_252',
       u'feat_253', u'feat_254', u'feat_255', u'feat_256', u'gap'],
      dtype='object', length=258)
Index([u'Id', u'smiles', u'feat_001', u'feat_002', u'feat_003', u'feat_004',
       u'feat_005', u'feat_006', u'feat_007', u'feat_008',
       ...
       u'feat_247', u'feat_248', u'feat_249', u'feat_250', u'feat_251',
       u'feat_252', u'feat_253', u'feat_254', u'feat_255', u'feat_256'],
      dtype='object', length=258)
Train features: (1000000, 94)
Train gap: (1000000,)
Test features: (824230, 94)


In [11]:
# Random Forest Prediction
RF = RandomForestRegressor(n_estimators=40, min_samples_split=8)
RF.fit(X_train, Y_train)

RF_pred = RF.predict(X_test)

In [12]:
RF_predred.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [ ]:
write_to_file("RF.csv", RF_pred)